In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
n = 63
N = 3
def f(t, x):
    return (2*N + 3)/(N + 2) * (n - 58) * np.sin((n - 58)*(3*(N + 7) + 3)/(N+7)/t * x)

c = N # u(a) = c
a = 0.2
b = 2.2
h = 0.05

In [ ]:
def plot_graphs(args: list[tuple[np.ndarray, np.ndarray]], names: list[str]):
    plt.figure(figsize=(10, 5))
    plt.xlabel("t")
    plt.ylabel("x(t)")
    plt.title("Численное решение уравнения")
    for i, el in enumerate(args):
        plt.plot(el[0], el[1], label=names[i])
    plt.legend()
    plt.grid()
    plt.show()

### Рунге-Кутта 4 порядка

In [ ]:
def runge_kutta_4(f, t0, x0, h, t_end):
    t_values = np.arange(t0, t_end + h, h)
    x_values = np.zeros(len(t_values))
    x_values[0] = x0

    for i in range(1, len(t_values)):
        t = t_values[i-1]
        x = x_values[i-1]

        k1 = h * f(t, x)
        k2 = h * f(t + h/2, x + k1/2)
        k3 = h * f(t + h/2, x + k2/2)
        k4 = h * f(t + h, x + k3)

        x_values[i] = x + (k1 + 2*k2 + 2*k3 + k4) / 6

    return t_values, x_values

In [ ]:
t_rk, x_rk = runge_kutta_4(f, a, c, h, b)


In [ ]:
def adams_bashforth_4(f, t0, x0, h, t_end):
    t_values = np.arange(t0, t_end + h, h)
    x_values = np.zeros(len(t_values))
    x_values[:4] = runge_kutta_4(f, t0, x0, h, t0 + 3*h)[1]

    for i in range(3, len(t_values) - 1):
        t = t_values[i]
        x_values[i+1] = x_values[i] + h * (55*f(t_values[i], x_values[i]) - 59*f(t_values[i-1], x_values[i-1]) +
                                           37*f(t_values[i-2], x_values[i-2]) - 9*f(t_values[i-3], x_values[i-3])) / 24

    return t_values, x_values

# Метод прогноза-коррекции (предиктор-корректор)
def predictor_corrector_4(f, t0, x0, h, t_end):
    t_values = np.arange(t0, t_end + h, h)
    x_values = np.zeros(len(t_values))
    x_values[:4] = runge_kutta_4(f, t0, x0, h, t0 + 3*h)[1]  # Первые 4 значения методом РК-4

    for i in range(3, len(t_values) - 1):
        t = t_values[i]

        # Предиктор (Адамс-Башфорт)
        x_pred = x_values[i] + h * (55*f(t_values[i], x_values[i]) - 59*f(t_values[i-1], x_values[i-1]) +
                                    37*f(t_values[i-2], x_values[i-2]) - 9*f(t_values[i-3], x_values[i-3])) / 24

        # Корректор (Адамса-Мултона)
        x_values[i+1] = x_values[i] + h * (9*f(t_values[i+1], x_pred) + 19*f(t_values[i], x_values[i]) -
                                           5*f(t_values[i-1], x_values[i-1]) + f(t_values[i-2], x_values[i-2])) / 24

    return t_values, x_values

# Решение методами
t_ab, x_ab = adams_bashforth_4(f, a, c, h, b)
t_pc, x_pc = predictor_corrector_4(f, a, c, h, b)

# Оценка погрешности по правилу Рунге (уменьшаем шаг в 2 раза)
h_half = h / 2
t_rk_half, x_rk_half = runge_kutta_4(f, a, c, h_half, b)

# Интерполяция решения с половинным шагом на исходные узлы
x_rk_half_interp = x_rk_half[::2]

# Оценка погрешности Рунге-Кутты
error_runge = np.sum(np.abs(x_rk - x_rk_half_interp)) / (2**4 - 1)
print(error_runge)

In [ ]:
plot_graphs([(t_rk, x_rk), (t_ab, x_ab), (t_pc, x_pc)], ["Рунге-Кутта 4-го порядка","Адамс-Башфорт (4-шаговый)", "Прогноз-коррекция (4 порядок)"])